# 3. Chain-of-Thought (CoT) Prompting - Razonamiento Paso a Paso

## Objetivos de Aprendizaje
- Comprender el concepto de Chain-of-Thought (CoT) prompting
- Implementar CoT con y sin ejemplos
- Aplicar CoT a problemas complejos y razonamiento lógico
- Combinar CoT con otras técnicas de prompting

## ¿Qué es Chain-of-Thought?

Chain-of-Thought (CoT) es una técnica que hace que el modelo "**piense en voz alta**" mostrando su proceso de razonamiento paso a paso antes de llegar a la respuesta final.

### Principio Básico:
En lugar de saltar directamente a la respuesta, el modelo:
1. **Descompone** el problema en pasos
2. **Razona** cada paso explícitamente  
3. **Construye** hacia la solución final
4. **Proporciona** la respuesta con justificación

### Ventajas:
- **Mejor precisión**: Especialmente en problemas complejos
- **Transparencia**: Puedes ver el razonamiento
- **Debugging**: Identificar dónde falla el razonamiento
- **Confianza**: Mayor seguridad en la respuesta

### Casos de Uso Ideales:
- Problemas matemáticos
- Razonamiento lógico
- Análisis complejos
- Toma de decisiones
- Resolución de problemas multi-paso

In [ ]:
# Configuración inicial
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import os
import time

# Configurar el modelo
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("GITHUB_TOKEN"),
    model="gpt-4",
    temperature=0.1  # Baja temperatura para razonamiento más consistente
)

print("✓ Modelo configurado para Chain-of-Thought")
print("✓ Temperature baja para razonamiento consistente")

✓ Modelo configurado para Chain-of-Thought
✓ Temperature baja para razonamiento consistente


## Comparación: Sin CoT vs Con CoT

Veamos la diferencia dramática que puede hacer CoT en problemas complejos.

In [2]:
# Comparación directa: razonamiento directo vs CoT
def comparar_sin_vs_con_cot():
    print("=== COMPARACIÓN: SIN CoT vs CON CoT ===")
    
    # Problema complejo que requiere múltiples pasos
    problema = """Una tienda tiene una promoción: 'Compra 2 productos y obtén 30% de descuento en el más barato'. 
    Juan compra una camiseta de 45€, unos zapatos de 120€ y una chaqueta de 80€. 
    ¿Cuánto paga en total?"""
    
    # Prompt sin CoT
    prompt_sin_cot = f"""Resuelve este problema:
    
{problema}
    
Respuesta:"""
    
    # Prompt con CoT
    prompt_con_cot = f"""Resuelve este problema paso a paso:
    
{problema}
    
Piensa paso a paso:
1. Primero identifica los productos y precios
2. Determina cómo se aplica la promoción
3. Calcula el descuento
4. Calcula el total final
    
Razonamiento:"""
    
    # Probar sin CoT
    print("\n1. SIN CHAIN-OF-THOUGHT:")
    print("-" * 30)
    try:
        response_sin = llm.invoke([HumanMessage(content=prompt_sin_cot)])
        print(response_sin.content)
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n" + "="*60)
    
    # Probar con CoT
    print("\n2. CON CHAIN-OF-THOUGHT:")
    print("-" * 30)
    try:
        response_con = llm.invoke([HumanMessage(content=prompt_con_cot)])
        print(response_con.content)
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n=== ANÁLISIS ===")
    print("• Sin CoT: Puede llegar a respuesta incorrecta o saltar pasos")
    print("• Con CoT: Muestra razonamiento completo y reduce errores")
    print("• CoT especialmente útil para problemas multi-paso")

# Ejecutar comparación
comparar_sin_vs_con_cot()

=== COMPARACIÓN: SIN CoT vs CON CoT ===

1. SIN CHAIN-OF-THOUGHT:
------------------------------
Error: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 22611 seconds before retrying.', 'details': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 22611 seconds before retrying.'}}


2. CON CHAIN-OF-THOUGHT:
------------------------------
Error: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 22609 seconds before retrying.', 'details': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 22609 seconds before retrying.'}}

=== ANÁLISIS ===
• Sin CoT: Puede llegar a respuesta incorrecta o saltar pasos
• Con CoT: Muestra razonamiento completo y reduce errores
• CoT especialmente útil para problemas multi-paso


## Zero-Shot Chain-of-Thought

La forma más simple de CoT: simplemente pedirle al modelo que "piense paso a paso".

In [ ]:
# Zero-shot CoT: solo agregar "piensa paso a paso"
def zero_shot_cot():
    print("=== ZERO-SHOT CHAIN-OF-THOUGHT ===")
    
    problemas = [
        "Si un tren viaja a 80 km/h y necesita llegar a una ciudad que está a 240 km, pero se detiene 15 minutos en una estación intermedia, ¿cuánto tiempo total toma el viaje?",
        "Una empresa tiene 150 empleados. El 40% trabaja en desarrollo, el 25% en ventas, el 20% en marketing y el resto en administración. Si cada empleado de desarrollo gana 50,000€ anuales, ¿cuál es el costo anual solo del departamento de desarrollo?",
        "María tiene el triple de edad que su hermana Ana. En 5 años, María tendrá el doble de la edad que tendrá Ana. ¿Cuántos años tiene cada una ahora?"
    ]
    
    for i, problema in enumerate(problemas, 1):
        print(f"\n{i}. PROBLEMA:")
        print(f"{problema}")
        
        # Prompt zero-shot CoT
        prompt = f"{problema}\n\nPiensa paso a paso:"
        
        try:
            response = llm.invoke([HumanMessage(content=prompt)])
            print("\nSOLUCIÓN:")
            print(response.content)
            
            # Análisis básico del razonamiento
            pasos = response.content.count('\n')
            tiene_calculo = any(op in response.content for op in ['+', '-', '*', '/', '=', '%'])
            
            print(f"\nAnálisis: {pasos} líneas de razonamiento, {'con' if tiene_calculo else 'sin'} cálculos explícitos")
            
        except Exception as e:
            print(f"Error: {e}")
        
        print("-" * 80)

# Ejecutar zero-shot CoT
zero_shot_cot()

## Few-Shot Chain-of-Thought

Combinamos CoT con ejemplos para mostrar el patrón de razonamiento deseado.

In [ ]:
# Few-shot CoT: ejemplos con razonamiento paso a paso
def few_shot_cot():
    print("=== FEW-SHOT CHAIN-OF-THOUGHT ===")
    
    # Nuevo problema para resolver
    nuevo_problema = "Un parking cobra 3€ la primera hora y 2€ cada hora adicional. Si alguien paga 15€, ¿cuántas horas estuvo estacionado?"
    
    # Prompt con ejemplos de razonamiento
    prompt_few_shot_cot = f"""Resuelve problemas matemáticos mostrando el razonamiento paso a paso:
    
Problema: Una pizza cuesta 12€ y cada ingrediente extra cuesta 1.50€. Si Pedro paga 18€, ¿cuántos ingredientes extra pidió?
Razonamiento:
1. Precio base de la pizza: 12€
2. Total pagado: 18€
3. Dinero gastado en extras: 18€ - 12€ = 6€
4. Costo por ingrediente extra: 1.50€
5. Número de ingredientes: 6€ ÷ 1.50€ = 4 ingredientes
Respuesta: Pedro pidió 4 ingredientes extra.
    
Problema: En una clase hay 24 estudiantes. Si se forman grupos de 6 estudiantes cada uno, ¿cuántos grupos se pueden formar? Si sobra algún estudiante, ¿cuántos?
Razonamiento:
1. Total de estudiantes: 24
2. Estudiantes por grupo: 6
3. División: 24 ÷ 6 = 4 grupos exactos
4. Verificación: 4 × 6 = 24 estudiantes
5. Resto: 24 - 24 = 0 estudiantes sobran
Respuesta: Se pueden formar 4 grupos completos y no sobra ningún estudiante.
    
Problema: {nuevo_problema}
Razonamiento:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_few_shot_cot)])
        print("PROBLEMA A RESOLVER:")
        print(nuevo_problema)
        print("\nSOLUCIÓN CON RAZONAMIENTO:")
        print(response.content)
        
        # Verificar si siguió el patrón
        razonamiento = response.content
        tiene_pasos_numerados = bool(re.search(r'\d+\.', razonamiento))
        tiene_calculos = any(op in razonamiento for op in ['=', '+', '-', '*', '/', '€'])
        tiene_respuesta_final = 'respuesta' in razonamiento.lower()
        
        print("\n=== ANÁLISIS DEL PATRÓN ===")
        print(f"✓ Pasos numerados: {'Sí' if tiene_pasos_numerados else 'No'}")
        print(f"✓ Cálculos explícitos: {'Sí' if tiene_calculos else 'No'}")
        print(f"✓ Respuesta final clara: {'Sí' if tiene_respuesta_final else 'No'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar few-shot CoT
import re
few_shot_cot()

## CoT para Razonamiento Lógico

Chain-of-Thought es especialmente poderoso para problemas de lógica y deducción.

In [ ]:
# CoT para problemas de lógica
def cot_razonamiento_logico():
    print("=== CoT PARA RAZONAMIENTO LÓGICO ===")
    
    # Problema de lógica clásico
    problema_logica = """En una mesa redonda se sientan 5 personas: Ana, Bruno, Carlos, Diana y Elena.
    - Ana no está al lado de Bruno
    - Carlos está exactamente frente a Diana
    - Elena está al lado derecho de Ana
    - Bruno está al lado de Carlos
    
¿Cuál es la disposición completa alrededor de la mesa?"""
    
    prompt_logica = f"""Resuelve este problema de lógica paso a paso:
    
{problema_logica}
    
Razona sistemáticamente:
1. Identifica las restricciones
2. Establece relaciones conocidas
3. Deduce posiciones paso a paso
4. Verifica que se cumplan todas las condiciones
    
Proceso de deducción:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_logica)])
        print("PROBLEMA DE LÓGICA:")
        print(problema_logica)
        print("\nPROCESO DE RAZONAMIENTO:")
        print(response.content)
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar razonamiento lógico
cot_razonamiento_logico()

In [ ]:
# CoT para análisis de casos complejos
def cot_analisis_complejo():
    print("=== CoT PARA ANÁLISIS COMPLEJO ===")
    
    # Caso de negocio complejo
    caso_negocio = """Una startup de software tiene las siguientes métricas:
    - 10,000 usuarios activos mensuales
    - Tasa de conversión a premium: 5%
    - Precio premium: 29€/mes
    - Costo de adquisición por usuario: 15€
    - Retención mensual: 85%
    - Costos operativos mensuales: 12,000€
    
La empresa está considerando reducir el precio a 19€/mes para aumentar la conversión a 8%. 
¿Es una buena decisión financiera?"""
    
    prompt_analisis = f"""Analiza este caso de negocio paso a paso:
    
{caso_negocio}
    
Estructura tu análisis:
1. Calcula métricas del escenario actual
2. Calcula métricas del escenario propuesto
3. Compara ingresos y costos
4. Considera factores adicionales
5. Proporciona recomendación fundamentada
    
Análisis detallado:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_analisis)])
        print("CASO DE NEGOCIO:")
        print(caso_negocio)
        print("\nANÁLISIS PASO A PASO:")
        print(response.content)
        
        # Verificar completitud del análisis
        analisis = response.content.lower()
        elementos_clave = [
            'ingresos', 'costos', 'beneficio', 'actual', 'propuesto', 
            'recomendación', 'conversión', 'usuarios'
        ]
        elementos_presentes = sum(1 for elemento in elementos_clave if elemento in analisis)
        
        print(f"\n=== COMPLETITUD DEL ANÁLISIS ===")
        print(f"✓ Elementos clave cubiertos: {elementos_presentes}/{len(elementos_clave)}")
        print(f"✓ Análisis completo: {'Sí' if elementos_presentes >= 6 else 'Parcial'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar análisis complejo
cot_analisis_complejo()

## Técnicas Avanzadas de CoT

In [ ]:
# Técnica 1: CoT con Auto-Verificación
def cot_con_verificacion():
    print("=== CoT CON AUTO-VERIFICACIÓN ===")
    
    problema = "Una piscina se llena con dos bombas. La bomba A la llena en 4 horas, la bomba B en 6 horas. Si funcionan juntas, ¿en cuánto tiempo llenan la piscina?"
    
    prompt_verificacion = f"""Resuelve este problema paso a paso y luego verifica tu respuesta:
    
{problema}
    
PASO 1 - RESOLUCIÓN:
Piensa paso a paso para encontrar la solución.
    
PASO 2 - VERIFICACIÓN:
Revisa tu cálculo usando un método diferente o verificando que los números tienen sentido.
    
PASO 3 - RESPUESTA FINAL:
Confirma tu respuesta final.
    
Proceso completo:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_verificacion)])
        print("PROBLEMA:")
        print(problema)
        print("\nSOLUCIÓN CON AUTO-VERIFICACIÓN:")
        print(response.content)
        
        # Verificar estructura
        contenido = response.content.lower()
        tiene_resolucion = 'paso 1' in contenido or 'resolución' in contenido
        tiene_verificacion = 'paso 2' in contenido or 'verificación' in contenido
        tiene_final = 'paso 3' in contenido or 'final' in contenido
        
        print("\n=== ESTRUCTURA ===")
        print(f"✓ Resolución: {'Sí' if tiene_resolucion else 'No'}")
        print(f"✓ Verificación: {'Sí' if tiene_verificacion else 'No'}")
        print(f"✓ Respuesta final: {'Sí' if tiene_final else 'No'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar CoT con verificación
cot_con_verificacion()

In [ ]:
# Técnica 2: CoT Multi-Perspectiva
def cot_multi_perspectiva():
    print("=== CoT MULTI-PERSPECTIVA ===")
    
    dilema = """Una empresa de delivery está considerando implementar un algoritmo de IA 
    para optimizar rutas que podría reducir costos en 20% pero eliminaría 100 empleos 
    de repartidores. ¿Debería implementarlo?"""
    
    prompt_multi = f"""Analiza este dilema desde múltiples perspectivas:
    
{dilema}
    
Analiza paso a paso desde cada perspectiva:
    
PERSPECTIVA 1 - FINANCIERA:
- Beneficios económicos
- Costos de implementación
- ROI a corto y largo plazo
    
PERSPECTIVA 2 - SOCIAL/ÉTICA:
- Impacto en empleados
- Responsabilidad social corporativa
- Percepción pública
    
PERSPECTIVA 3 - ESTRATÉGICA:
- Competitividad del mercado
- Innovación tecnológica
- Sostenibilidad del negocio
    
SÍNTESIS:
- Integra las perspectivas
- Propone soluciones alternativas
- Recomienda curso de acción
    
Análisis completo:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_multi)])
        print("DILEMA EMPRESARIAL:")
        print(dilema)
        print("\nANÁLISIS MULTI-PERSPECTIVA:")
        print(response.content)
        
        # Verificar cobertura de perspectivas
        analisis = response.content.lower()
        perspectivas = ['financiera', 'social', 'ética', 'estratégica', 'síntesis']
        perspectivas_cubiertas = sum(1 for p in perspectivas if p in analisis)
        
        print(f"\n=== COBERTURA DEL ANÁLISIS ===")
        print(f"✓ Perspectivas cubiertas: {perspectivas_cubiertas}/{len(perspectivas)}")
        print(f"✓ Análisis integral: {'Sí' if perspectivas_cubiertas >= 4 else 'Parcial'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar análisis multi-perspectiva
cot_multi_perspectiva()

## CoT para Debugging de Código

Una aplicación práctica muy útil: usar CoT para analizar y debuggear código.

In [ ]:
# CoT para debugging de código
def cot_debugging():
    print("=== CoT PARA DEBUGGING DE CÓDIGO ===")
    
    codigo_con_bug = '''def calcular_promedio(numeros):
    total = 0
    for numero in numeros:
        total += numero
    promedio = total / len(numeros)
    return promedio

# Uso
datos = []
resultado = calcular_promedio(datos)
print(f"El promedio es: {resultado}")'''
    
    prompt_debugging = f"""Analiza este código paso a paso para encontrar problemas:
    
```python
{codigo_con_bug}
```
    
PASO 1 - COMPRENSIÓN:
¿Qué se supone que hace este código?
    
PASO 2 - ANÁLISIS LÍNEA POR LÍNEA:
Examina cada línea en busca de problemas potenciales.
    
PASO 3 - IDENTIFICACIÓN DE PROBLEMAS:
¿Qué errores o problemas específicos encuentras?
    
PASO 4 - CASOS PROBLEMÁTICOS:
¿En qué situaciones fallaría este código?
    
PASO 5 - SOLUCIÓN:
¿Cómo arreglarías estos problemas?
    
Análisis de debugging:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_debugging)])
        print("CÓDIGO A ANALIZAR:")
        print(codigo_con_bug)
        print("\nANÁLISIS DE DEBUGGING:")
        print(response.content)
        
        # Verificar si encontró el problema principal
        analisis = response.content.lower()
        encontro_division_cero = any(term in analisis for term in ['división', 'cero', 'vacía', 'empty'])
        propuso_solucion = 'if' in analisis or 'len(' in analisis or 'excepción' in analisis
        
        print(f"\n=== EFECTIVIDAD DEL DEBUGGING ===")
        print(f"✓ Identificó división por cero: {'Sí' if encontro_division_cero else 'No'}")
        print(f"✓ Propuso solución: {'Sí' if propuso_solucion else 'No'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar debugging
cot_debugging()

## Limitaciones y Consideraciones de CoT

### ✅ Cuándo Usar CoT:
- Problemas matemáticos complejos
- Razonamiento lógico multi-paso
- Análisis que requiere transparencia
- Cuando necesitas verificar el proceso
- Problemas donde el "por qué" es importante

### ⚠️ Limitaciones:
- **Más tokens**: Respuestas más largas = mayor costo
- **Tiempo**: Razonamiento paso a paso toma más tiempo
- **Complejidad innecesaria**: Para problemas simples puede ser excesivo
- **Razonamiento erróneo**: Puede mostrar lógica incorrecta convincente

### 🎯 Mejores Prácticas:
1. **Estructura clara**: Define pasos específicos
2. **Verificación**: Incluye auto-verificación cuando sea posible
3. **Ejemplos**: Usa few-shot para mostrar patrón deseado
4. **Temperatura baja**: Para razonamiento más consistente
5. **Validación externa**: Verifica respuestas críticas independientemente

In [ ]:
# Ejercicio final: Diseña tu propio CoT prompt
def ejercicio_cot():
    print("=== EJERCICIO: DISEÑA TU CoT PROMPT ===")
    print("\nTarea: Crear un sistema CoT para análisis de inversión")
    print("\nEscenario:")
    print("Una persona tiene 10,000€ para invertir y está considerando tres opciones:")
    print("1. Acciones de tech (retorno esperado 12% anual, riesgo alto)")
    print("2. Bonos gubernamentales (retorno 3% anual, riesgo bajo)")
    print("3. Fondo mixto (retorno 7% anual, riesgo medio)")
    print("\nLa persona es joven (25 años) y puede asumir riesgo moderado.")
    
    # Template para el estudiante
    template_cot = """
    # TU PROMPT CoT AQUÍ:
    
    Analiza esta decisión de inversión paso a paso:
    
    [ESCENARIO]
    
    PASO 1 - PERFIL DEL INVERSOR:
    - Analiza edad, tolerancia al riesgo, horizonte temporal
    
    PASO 2 - ANÁLISIS DE OPCIONES:
    - Evalúa cada opción: retorno, riesgo, liquidez
    
    PASO 3 - ESTRATEGIA DE DIVERSIFICACIÓN:
    - Considera combinar opciones
    
    PASO 4 - RECOMENDACIÓN:
    - Proporciona recomendación específica con justificación
    
    Análisis de inversión:
    """
    
    print("\nDiseña un prompt CoT estructurado:")
    print(template_cot)
    
    # Prompt de ejemplo bien diseñado
    prompt_ejemplo = """Analiza esta decisión de inversión usando razonamiento paso a paso:
    
SITUACIÓN:
Inversor de 25 años con 10,000€ considerando:
- Acciones tech: 12% retorno anual, riesgo alto
- Bonos: 3% retorno anual, riesgo bajo  
- Fondo mixto: 7% retorno anual, riesgo medio
Tolerancia: riesgo moderado
    
PASO 1 - PERFIL DEL INVERSOR:
Analiza edad, horizonte temporal y tolerancia al riesgo.
    
PASO 2 - EVALUACIÓN DE OPCIONES:
Para cada opción, calcula:
- Valor esperado en 10 años
- Nivel de riesgo vs. perfil
- Pros y contras específicos
    
PASO 3 - ESTRATEGIA DE PORTFOLIO:
Considera distribución óptima entre opciones basada en:
- Diversificación de riesgo
- Maximización de retorno ajustado por riesgo
- Liquidez y flexibilidad
    
PASO 4 - RECOMENDACIÓN FINAL:
Proporciona distribución específica (porcentajes) con:
- Justificación detallada
- Proyección a 10 años
- Consideraciones adicionales
    
Análisis completo:"""
    
    print("\n=== PROMPT DE REFERENCIA ===")
    print(prompt_ejemplo)
    
    # Ejecutar el prompt ejemplo
    print("\n=== RESULTADO DEL ANÁLISIS ===")
    try:
        response = llm.invoke([HumanMessage(content=prompt_ejemplo)])
        print(response.content)
        
        # Análisis de la respuesta
        analisis = response.content.lower()
        elementos = ['perfil', 'evaluación', 'portfolio', 'recomendación', '%', '€', 'años']
        elementos_presentes = sum(1 for elem in elementos if elem in analisis)
        
        print(f"\n=== CALIDAD DEL ANÁLISIS ===")
        print(f"✓ Elementos clave: {elementos_presentes}/{len(elementos)}")
        print(f"✓ Análisis completo: {'Sí' if elementos_presentes >= 5 else 'Parcial'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar ejercicio
ejercicio_cot()

## Conceptos Clave Aprendidos

1. **Chain-of-Thought** mejora dramáticamente la precisión en problemas complejos
2. **Zero-shot CoT** es tan simple como agregar "piensa paso a paso"
3. **Few-shot CoT** combina ejemplos con razonamiento para mejor control
4. **Estructura explícita** guía el razonamiento hacia análisis completos
5. **Auto-verificación** aumenta la confiabilidad de las respuestas

## Próximos Pasos

En el siguiente notebook exploraremos **Técnicas Avanzadas** como Tree of Thoughts (ToT), Self-Consistency, y otras metodologías cutting-edge que llevan el prompt engineering al siguiente nivel.

### Para Practicar:
1. Aplica CoT a problemas de tu dominio específico
2. Experimenta con diferentes niveles de estructura
3. Compara precisión con y sin CoT
4. Desarrolla patrones de verificación personalizados